This notebook implements some of the theoretical concepts discussed in Lecture 8 of the course "An introduction to quantum entanglement", available at https://ion.nechita.net/2020-an-introduction-to-quantum-entanglement/

Please send remarks and comments regarding both this notebook and the lecture notes to nechita@irsamc.ups-tlse.fr

In [81]:
import numpy as np
import scipy
import qutip

# The "tiles" PPT entangled state

In [79]:
# the 5 vectors of the "tiles" UPB for 2 qutrits
x1 = np.kron([[1, 0, 0]], [[1, -1, 0]]/np.sqrt(2))
x2 = np.kron([[0, 0, 1]], [[0, 1, -1]]/np.sqrt(2))
x3 = np.kron([[1, -1, 0]]/np.sqrt(2), [[0, 0, 1]])
x4 = np.kron([[0, 1, -1]]/np.sqrt(2), [[1, 0, 0]])
x5 = np.kron([[1, 1, 1]], [[1, 1, 1]])/3

# the "tiles" density matrix
rhoX = 1/(9-5)*(np.identity(9) - (x1.T@x1 + x2.T@x2 + x3.T@x3 + x4.T@x4 + x5.T@x5))

# it has indeed positive eigenvalues
print("Eigenvalues of rhoX : ", scipy.linalg.eigh(rhoX, eigvals_only=True))

Eigenvalues of rhoX :  [3.63165655e-18 1.74018794e-17 7.40004164e-17 9.38320119e-17
 1.13224561e-16 2.50000000e-01 2.50000000e-01 2.50000000e-01
 2.50000000e-01]


In [88]:
# we are dealing with 2 qutrits
d = 3

# compute the partial transpose with respect to the second subsystem
rhoXq = qutip.Qobj(rhoX)
rhoXq.dims = [[d,d], [d,d]]
rhoXGammaq = qutip.partial_transpose(rhoXq, [0, 1])
rhoXGamma = rhoXGammaq.full()

# test if it is positive semidefinite
print("Minimal eigenvalue of the partial transpose = ", np.min(scipy.linalg.eigh(rhoXGamma, eigvals_only=True)))

Minimal eigenvalue of the partial transpose =  -6.2074201300892115e-18


Hence, the quantum "tiles" state $\rho_X$ is PPT. One can show, using the properties of the UPB $X$, that $\rho_X$ is entangled.

# Semidefinite programs

We are using the package **CVXPY** to easily code SDPs in python, with the ***CVXOPT*** solver. We are discussing two important examples here, the operator norm and the trace norm of matrices.

In [56]:
import cvxpy as cp
print(cp.installed_solvers())

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCS']


As an easy example, recall the SDP for the operator norm: 
$\min t$ under the constraints $-t I_d \leq A \leq tI_d$. We are comparing the value of the SDP with the value returned by the singular value decomposition, for a random matrix. 

In [89]:
n = 100

# construct a random n x n matrix 
A = np.random.randn(n, n)
A = (A+A.T)/2;

# construct the problem
t = cp.Variable(1)
objective = cp.Minimize(t)
constraints = [t*np.identity(n) + A >> 0, t*np.identity(n) - A >> 0]
prob = cp.Problem(objective, constraints)

# solve the problem
prob.solve(solver="CVXOPT")

print("SDP value = ", prob.value)
print("Norm of the matrix = ", np.max(scipy.linalg.eigh(A, eigvals_only=True)))

SDP value =  13.961380533208963
Norm of the matrix =  13.321719970830118


# The DPS hierarchy

We need the following implementation of the partial trace function, which respects the structure of **CVXPY** SDPs

In [102]:
from cvxpy.expressions.expression import Expression


def expr_as_np_array(cvx_expr):
    if cvx_expr.is_scalar():
        return np.array(cvx_expr)
    elif len(cvx_expr.shape) == 1:
        return np.array([v for v in cvx_expr])
    else:
        # then cvx_expr is a 2d array
        rows = []
        for i in range(cvx_expr.shape[0]):
            row = [cvx_expr[i,j] for j in range(cvx_expr.shape[1])]
            rows.append(row)
        arr = np.array(rows)
        return arr


def np_array_as_expr(np_arr):
    aslist = np_arr.tolist()
    expr = cp.bmat(aslist)
    return expr


def np_partial_trace(rho, dims, axis=0):
    """
    Takes partial trace over the subsystem defined by 'axis'
    rho: a matrix
    dims: a list containing the dimension of each subsystem
    axis: the index of the subsytem to be traced out
    (We assume that each subsystem is square)
    """
    dims_ = np.array(dims)
    # Reshape the matrix into a tensor with the following shape:
    # [dim_0, dim_1, ..., dim_n, dim_0, dim_1, ..., dim_n]
    # Each subsystem gets one index for its row and another one for its column
    reshaped_rho = np.reshape(rho, np.concatenate((dims_, dims_), axis=None))

    # Move the subsystems to be traced towards the end
    reshaped_rho = np.moveaxis(reshaped_rho, axis, -1)
    reshaped_rho = np.moveaxis(reshaped_rho, len(dims)+axis-1, -1)

    # Trace over the very last row and column indices
    traced_out_rho = np.trace(reshaped_rho, axis1=-2, axis2=-1)

    # traced_out_rho is still in the shape of a tensor
    # Reshape back to a matrix
    dims_untraced = np.delete(dims_, axis)
    rho_dim = np.prod(dims_untraced)
    return traced_out_rho.reshape([rho_dim, rho_dim])


def partial_trace(rho, dims, axis=0):
    if not isinstance(rho, Expression):
        rho = cp.Constant(shape=rho.shape, value=rho)
    rho_np = expr_as_np_array(rho)
    traced_rho = np_partial_trace(rho_np, dims, axis)
    traced_rho = np_array_as_expr(traced_rho)
    return traced_rho

We shall consider a noisy version of the "tiles" state:
$$\rho_p = (1-p)\rho_X + p \frac{I_9}{9}.$$
Let us set $p=0.1$. 

In [90]:
# a noisy version of the "tiles" state
p=0.1
rho = (1-p)*rhoX + p*np.identity(9)/9

# the PPT criterion does not detect the entanglement in rho
rhoq = qutip.Qobj(rho)
rhoq.dims = [[d,d], [d,d]]
rhoGammaq = qutip.partial_transpose(rhoq, [0, 1])
rhoGamma = rhoGammaq.full()

print("Minimal eigenvalue of the partial transpose = ", np.min(scipy.linalg.eigh(rhoXGamma, eigvals_only=True)))

Minimal eigenvalue of the partial transpose =  -6.2074201300892115e-18


We check the existence of a $2$-symmetric extension 

In [108]:
# construct the problem; first, the variable
sigma = cp.Variable((d**3,d**3), symmetric=True)

# then, the constraints: postiivity and marginals
constraints = [sigma >> 0]
constraints += [partial_trace(sigma, dims = [d, d, d], axis = 2) == rho]
constraints += [partial_trace(sigma, dims = [d, d, d], axis = 1) == rho]

# we minimize the trace of sigma
prob = cp.Problem(cp.Maximize(cp.trace(sigma)), constraints)

# solve the problem
prob.solve(solver="CVXOPT")

print("SDP value = ", prob.value)

SDP value =  1.0000000000000002


The state $\rho$ has a $2$-symmetric extension $\implies$ we cannot conclude

We check the existence of a $3$-symmetric extension 

In [ ]:
# construct the problem; first, the variable
sigma = cp.Variable((d**4,d**4), symmetric=True)

# then, the constraints: postiivity and marginals
constraints = [sigma >> 0]
constraints += [partial_trace(partial_trace(sigma, dims = [d, d, d, d], axis = 3), dims = [d, d, d], axis = 2) == rho]
constraints += [partial_trace(partial_trace(sigma, dims = [d, d, d, d], axis = 3), dims = [d, d, d], axis = 1) == rho]
constraints += [partial_trace(partial_trace(sigma, dims = [d, d, d, d], axis = 1), dims = [d, d, d], axis = 1) == rho]

# we minimize the trace of sigma
prob = cp.Problem(cp.Maximize(cp.trace(sigma)), constraints)

# solve the problem
prob.solve(solver="CVXOPT", verbose = True)

print("SDP value = ", prob.value)